In [1]:
import tensorflow as tf
import numpy as np

from tensorflow import keras
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.efficientnet import preprocess_input

2023-12-20 22:09:27.654514: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [3]:
model = keras.models.load_model('./efficientnetb1_v1_49_0.691.h5')

2023-12-20 22:10:04.840323: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-12-20 22:10:04.847542: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-12-20 22:10:04.848169: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-12-20 22:10:04.849068: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the approp

In [7]:
# test_gen = ImageDataGenerator(preprocessing_function=preprocess_input)
# test_ds = test_gen.flow_from_directory(
#     './color-classifier/data/test',
#     target_size=(299, 299),
#     batch_size=32
# )

test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    './data/train/',
    image_size=(299, 299),
    batch_size=32,
    label_mode="categorical",
)

Found 6239 files belonging to 12 classes.


In [8]:
# return loss value (CategoricalCrossEntropy) and accuracy metric
model.evaluate(test_ds)

2023-12-20 22:10:56.931951: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8100
2023-12-20 22:10:57.545591: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-12-20 22:10:57.546101: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-12-20 22:10:57.546125: W tensorflow/stream_executor/gpu/asm_compiler.cc:80] Couldn't get ptxas version string: INTERNAL: Couldn't invoke ptxas --version
2023-12-20 22:10:57.546746: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-12-20 22:10:57.546841: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] INTERNAL: Failed to launch ptxas
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.


195/195 [==============================] - 34s 149ms/step - loss: 0.4734 - accuracy: 0.8724


[0.47343114018440247, 0.8724154233932495]

In [11]:
## Apply the model to random image sample
img_path = './data/test/1000000678.jpg'
img = load_img(img_path, target_size = (299, 299, 3))
img
x = np.array(img)
X = np.array([x])
X.shape

X = preprocess_input(X)
pred = model.predict(X)
# pred[0]

classes = ['black',
 'blue',
 'brown',
 'green',
 'grey',
 'orange',
 'pink',
 'purple',
 'red',
 'silver',
 'white',
 'yellow']

dict(zip(classes, pred[0]))

1/1 [==============================] - 0s 27ms/step


{'black': -6.610557,
 'blue': -3.8168256,
 'brown': -4.2920504,
 'green': 1.3504324,
 'grey': -4.747616,
 'orange': 3.850803,
 'pink': 0.101556234,
 'purple': -0.6918626,
 'red': -2.4467452,
 'silver': -5.657035,
 'white': -1.6534253,
 'yellow': 5.958621}

## Convert Keras to TF-lite model

In [12]:
model = keras.models.load_model('./efficientnetb1_v1_49_0.691.h5')

converter = tf.lite.TFLiteConverter.from_keras_model(model)

tflite_model = converter.convert()

# save the model for TF lite
with open('color-classifier-model.tflite', 'wb') as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmphpwkn10b/assets


INFO:tensorflow:Assets written to: /tmp/tmphpwkn10b/assets
2023-12-20 22:14:26.499889: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2023-12-20 22:14:26.499935: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2023-12-20 22:14:26.500666: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: /tmp/tmphpwkn10b
2023-12-20 22:14:26.582376: I tensorflow/cc/saved_model/reader.cc:81] Reading meta graph with tags { serve }
2023-12-20 22:14:26.582416: I tensorflow/cc/saved_model/reader.cc:122] Reading SavedModel debug info (if present) from: /tmp/tmphpwkn10b
2023-12-20 22:14:26.828610: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:354] MLIR V1 optimization pass is not enabled
2023-12-20 22:14:26.912441: I tensorflow/cc/saved_model/loader.cc:228] Restoring SavedModel bundle.
2023-12-20 22:14:27.935437: I tensorflow/cc/saved_model/loader.cc:212] Running initializatio

In [14]:
!ls -lh

total 402M
-rw-r--r-- 1 jovyan jovyan  26M Dec 20 22:14 color-classifier-model.tflite
-rw-rw-r-- 1 jovyan jovyan 681K Dec 20 22:08 color-classifier-model-training.ipynb
-rw-rw---- 1 jovyan jovyan 484K Dec 20 13:44 core.5530
drwxrwsr-x 4 jovyan jovyan 4.0K Dec 20 12:01 data
-rw-rw-r-- 1 jovyan jovyan  52M Dec 20 11:58 data.zip
-rw-rw-r-- 1 jovyan jovyan  28M Dec 20 13:41 efficientnetb1_v1_01_0.580.h5
-rw-rw-r-- 1 jovyan jovyan  28M Dec 20 13:41 efficientnetb1_v1_02_0.622.h5
-rw-rw-r-- 1 jovyan jovyan  28M Dec 20 13:42 efficientnetb1_v1_03_0.650.h5
-rw-rw-r-- 1 jovyan jovyan  28M Dec 20 13:42 efficientnetb1_v1_04_0.670.h5
-rw-rw-r-- 1 jovyan jovyan  28M Dec 20 13:43 efficientnetb1_v1_05_0.676.h5
-rw-rw-r-- 1 jovyan jovyan  28M Dec 20 13:45 efficientnetb1_v1_10_0.683.h5
-rw-rw-r-- 1 jovyan jovyan  28M Dec 20 14:08 efficientnetb1_v1_32_0.684.h5
-rw-rw-r-- 1 jovyan jovyan  28M Dec 20 14:10 efficientnetb1_v1_37_0.689.h5
-rw-rw-r-- 1 jovyan jovyan  28M Dec 20 14:11 efficientnetb1_v1_38_0.690.